In [41]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, widgets # интерактивные виджеты

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from mpl_toolkits.basemap import Basemap # карты

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from datetime import datetime
import warnings


In [2]:
# Для демонстрации будем использовать файл с прогнозами и фактом на июнь 2016 года
df = pd.read_csv('/Users/shrlq/Documents/python_yandex/course 6/week_6/prediction_jun3.csv',
                parse_dates=True, date_parser='histEndDay')
df.drop(columns='resid', inplace=True)
df.head()


,prediction,target,geoID,histEndDay,histEndHour,step
0,18.399837,26.0,1075,2016-05-31,23,1
1,100.900062,151.0,1076,2016-05-31,23,1
2,87.552605,98.0,1077,2016-05-31,23,1
3,108.196170,122.0,1125,2016-05-31,23,1
4,44.686587,27.0,1126,2016-05-31,23,1


In [3]:
df.index = [datetime.strptime(a[3] + ' ' + str(a[4]), '%Y-%m-%d %H') for ind, a in df.iterrows()]
df.head()

,prediction,target,geoID,histEndDay,histEndHour,step
2016-05-31 23:00:00,18.399837,26.0,1075,2016-05-31,23,1
2016-05-31 23:00:00,100.900062,151.0,1076,2016-05-31,23,1
2016-05-31 23:00:00,87.552605,98.0,1077,2016-05-31,23,1
2016-05-31 23:00:00,108.196170,122.0,1125,2016-05-31,23,1
2016-05-31 23:00:00,44.686587,27.0,1126,2016-05-31,23,1


In [184]:
# Функция для построения графика динамики прогноза и факта поездок для выбранного региона
def results_plot_1(geo, step):
    plt.figure(figsize=(20, 7))
    x = df.query('geoID == ' + str(geo) + 'and step == ' + str(step))['prediction']
    y = df.query('geoID == ' + str(geo) + 'and step == ' + str(step))['target']
    
    plt.plot(x, label='Прогноз' )
    plt.plot(y, alpha=.5, label='Факт')
    plt.title('Динамика количества поездок в июне 2016 года в Нью-Йорке')
    plt.legend()
    
    plt.show()

In [186]:
# Построим график динамики прогноза и факта поездок для выбранного региона
geos = df['geoID'].unique()
steps = np.arange(1, 7)
interact(results_plot_1, geo = geos, step = (1, 6, 1))

interactive(children=(Dropdown(description='geo', options=(1075, 1076, 1077, 1125, 1126, 1127, 1128, 1129, 113…

<function __main__.results_plot_1(geo, step)>

In [20]:
# Для построения карты с количеством поездок в каждой зоне, нам понадобится файл с координатами зон
regions_data = pd.read_csv('/Users/shrlq/Documents/python_yandex/course 6/week 1/regions.csv', sep = ';')
regions_data.rename(columns = {'region': 'geoID'}, inplace=True)
regions_data.head()

,geoID,west,east,south,north
0,1,-74.25559,-74.244478,40.496120,40.504508
1,2,-74.25559,-74.244478,40.504508,40.512896
2,3,-74.25559,-74.244478,40.512896,40.521285
3,4,-74.25559,-74.244478,40.521285,40.529673
4,5,-74.25559,-74.244478,40.529673,40.538061


In [58]:
# Объеденим координаты и данные о поездках
data = df.merge(regions_data)
data.head()

,prediction,target,geoID,histEndDay,histEndHour,step,west,east,south,north
0,18.399837,26.0,1075,2016-05-31,23,1,-74.022246,-74.011135,40.697437,40.705825
1,58.682979,30.0,1075,2016-06-01,0,1,-74.022246,-74.011135,40.697437,40.705825
2,14.536358,19.0,1075,2016-06-01,1,1,-74.022246,-74.011135,40.697437,40.705825
3,26.970712,39.0,1075,2016-06-01,2,1,-74.022246,-74.011135,40.697437,40.705825
4,87.980741,71.0,1075,2016-06-01,3,1,-74.022246,-74.011135,40.697437,40.705825


In [59]:
# Вычислим центр каждой зоны
lats_list = []
lons_list = []
for lab, row in data.iterrows():
    lat_mean = np.mean([row[6+2], row[6+3]])
    long_mean = np.mean([row[6+0], row[6+1]])
    lats_list.append(lat_mean)
    lons_list.append(long_mean)

data['lat_center'] = lats_list
data['long_center'] = lons_list

data.head()

,prediction,target,geoID,histEndDay,histEndHour,step,west,east,south,north,lat_center,long_center
0,18.399837,26.0,1075,2016-05-31,23,1,-74.022246,-74.011135,40.697437,40.705825,40.701631,-74.016691
1,58.682979,30.0,1075,2016-06-01,0,1,-74.022246,-74.011135,40.697437,40.705825,40.701631,-74.016691
2,14.536358,19.0,1075,2016-06-01,1,1,-74.022246,-74.011135,40.697437,40.705825,40.701631,-74.016691
3,26.970712,39.0,1075,2016-06-01,2,1,-74.022246,-74.011135,40.697437,40.705825,40.701631,-74.016691
4,87.980741,71.0,1075,2016-06-01,3,1,-74.022246,-74.011135,40.697437,40.705825,40.701631,-74.016691


In [222]:
# Функция для построения визуализации поездок на карте (по всем зонам)
def result_plot_2(histEndDay, histEndHour, step):
    warnings.filterwarnings('ignore')
    
    data_ = data.loc[(data['histEndDay'] == histEndDay) & 
                    (data['histEndHour'] == histEndHour) & 
                    (data['step'] == step), :]

    fig, axes = plt.subplots(1, 2, figsize=(20, 5))
    fig.suptitle ('Количество поездок в июне 2016 года в Нью-Йорке', fontsize=16)
    
    axes[0].set_title("Прогноз")
    m1 = Basemap(llcrnrlon=-74.1,llcrnrlat=40.62,urcrnrlon=-73.7,urcrnrlat=40.88, epsg=4269, ax=axes[0])
    m1.arcgisimage(service = "ESRI_Imagery_World_2D", xpixels = 1500, verbose= True)
    m1.scatter(data_['long_center'], data_['lat_center'], c=data_['prediction'], cmap='jet')
    plt.grid()
    
    axes[1].set_title("Факт")
    
    m2 = Basemap(llcrnrlon=-74.1,llcrnrlat=40.62,urcrnrlon=-73.7,urcrnrlat=40.88, epsg=4269, ax=axes[1])
    m2.arcgisimage(service = "ESRI_Imagery_World_2D", xpixels = 1500, verbose= True)
    m2.scatter(data_['long_center'], data_['lat_center'], c=data_['target'], cmap='jet')
    plt.grid()
    
    mappable = axes[1].collections[0]
    fig.colorbar(mappable=mappable, fraction=0.032, pad=0.01)
    
    plt.show()

In [223]:
# Для выбранного времени (конца истории) визуализируем кол-во поездок на карте - прогноз и факт
histEndDay = data['histEndDay'].unique()[1:                                        ]
histEndHour = np.arange(0, 25)
step = np.arange(1, 7)

interact(result_plot_2, histEndDay = histEndDay, histEndHour = histEndHour, 
         step = step)

interactive(children=(Dropdown(description='histEndDay', options=('2016-06-01', '2016-06-02', '2016-06-03', '2…

<function __main__.result_plot_2(histEndDay, histEndHour, step)>